In [24]:
!pip install fastbook

In [25]:
import fastbook
fastbook.setup_book()

In [26]:
#hide
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from IPython.display import Image, display_svg, SVG

In [27]:
import pandas as pd
from pathlib import Path

test_df = pd.read_csv(Path('/kaggle/input/playground-series-s4e8/test.csv'))
train_df = pd.read_csv(Path('/kaggle/input/playground-series-s4e8/train.csv'))
sample_df = pd.read_csv(Path('/kaggle/input/playground-series-s4e8/sample_submission.csv'))

In [28]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [29]:
# train_df.drop(['gill-spacing','cap-surface','gill-attachment','stem-root',
#          'stem-surface','veil-type','veil-color','spore-print-color'], inplace = True, axis =1)

In [30]:
# test_df.drop(['gill-spacing','cap-surface','gill-attachment','stem-root',
#          'stem-surface','veil-type','veil-color','spore-print-color'], inplace = True, axis =1)

In [31]:
# # Function to check the number of rows before and after applying a filter
# def filter_summary(df, column):
#     value_counts = df[column].value_counts()
#     value_keep = value_counts[value_counts>9].index
#     df_filtered = df[df[column].isin(value_keep)]
#     return df_filtered

# # Apply filters and summarize the impact
# train_df = filter_summary(train_df, 'cap-shape')
# train_df = filter_summary(train_df, 'cap-surface')
# train_df = filter_summary(train_df, 'cap-color')
# train_df = filter_summary(train_df, 'does-bruise-or-bleed')
# train_df = filter_summary(train_df, 'gill-attachment')
# train_df = filter_summary(train_df, 'gill-spacing')
# train_df = filter_summary(train_df, 'gill-color')
# train_df = filter_summary(train_df, 'stem-root')
# train_df = filter_summary(train_df, 'stem-surface')
# train_df = filter_summary(train_df, 'stem-color')
# train_df = filter_summary(train_df, 'has-ring')

In [32]:
len(train_df), len(test_df)

(3116945, 2077964)

In [33]:
train_df.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,4.51,15.39,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,4.79,6.48,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,6.85,9.93,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,4.16,6.53,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,3.37,8.36,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [34]:
len(train_df)

3116945

In [35]:
train_size = int(0.8 * len(train_df))
train_idx = list(range(train_size))
valid_idx = list(range(train_size, len(train_df)))

splits = (list(train_idx),list(valid_idx))

In [36]:
depVar = 'class'
train_df[depVar] = train_df[depVar].astype('category')
procs = [Categorify, FillMissing]
# FillMissing is just:
# df_train = df_train.fillna(df_train.mean())
continuous, categorical = cont_cat_split(train_df, 1, dep_var=depVar)
to = TabularPandas(train_df, procs, categorical, continuous, y_names=depVar, splits=splits)

/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [37]:
len(to.train), len(to.valid)

(2493556, 623389)

In [38]:
def rmse(prediction, label):
    return round(math.sqrt(((prediction-label)**2).mean()), 6)
def m_rmse(model, xs, y):
    return rmse(model.predict(xs), y)

In [39]:
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y
valid_xs.shape, valid_y.shape

((623389, 22), (623389,))

In [41]:
!pip install xgboost

In [42]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

modelXGB = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss')
modelXGB.fit(xs, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [43]:
valid_predictions = modelXGB.predict(valid_xs)
accuracy_score(valid_y, valid_predictions)

0.9804006807948167

In [45]:
# Reprocess test_df with the same pipeline
to_test = TabularPandas(test_df, procs, categorical, continuous)
test_xs = to_test.xs
test_xs = test_xs[xs.columns]

/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [46]:
# Make predictions
predictions = modelXGB.predict(test_xs)

# Create a new list with modified predictions
modified_predictions = ['p' if pred == 1 else 'e' for pred in predictions]

In [47]:
ids = test_xs['id'].to_list()
submission_csv = pd.DataFrame({
    'id': ids,
    'class': modified_predictions
})

In [48]:
submission_csv.head()

,id,class
0,3116945,e
1,3116946,e
2,3116947,p
3,3116948,e
4,3116949,e


In [50]:
submission_csv.to_csv('submission.csv', index=False)